In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Definir o período de 8 anos (01/01/2017 a 01/01/2025)
start_date = datetime(2017, 1, 1)
end_date = datetime(2025, 1, 1)
days = (end_date - start_date).days + 1  # 2923 dias

# Aumentar para 20.000 linhas para manter proporção de ~7 anúncios por dia
total_rows = 20000

# Criar lista de datas com 6-8 entradas por dia para maior variação
dates = []
for i in range(days):
    current_date = start_date + timedelta(days=i)
    entries_per_day = np.random.randint(6, 9)  # Variação maior
    dates.extend([current_date] * entries_per_day)

# Ajustar para exatamente 20.000 linhas
if len(dates) > total_rows:
    dates = dates[:total_rows]
elif len(dates) < total_rows:
    dates.extend([end_date] * (total_rows - len(dates)))

# Função para gerar campanhas sequenciais SEM SOBREPOSIÇÃO
def generate_campaigns(start_date, end_date):
    campaigns = []
    current_date = start_date
    campaign_id = 1

    while current_date < end_date:
        # Duração entre 60 e 180 dias (2 a 6 meses)
        duration_days = np.random.randint(60, 181)
        end_campaign = current_date + timedelta(days=duration_days)

        # Se a campanha ultrapassar o end_date, ajustar
        if end_campaign > end_date:
            end_campaign = end_date

        campaigns.append({
            'Campanha': f'Campanha_{campaign_id}',
            'Start': current_date,
            'End': end_campaign
        })

        # IMPORTANTE: Próxima campanha começa NO DIA SEGUINTE ao fim da anterior
        current_date = end_campaign + timedelta(days=1)
        campaign_id += 1

    return campaigns

# Gerar campanhas
campaigns_list = generate_campaigns(start_date, end_date)
campaigns_df = pd.DataFrame(campaigns_list)

print("Campanhas geradas:")
for camp in campaigns_list:
    duration = (camp['End'] - camp['Start']).days
    print(f"{camp['Campanha']}: {camp['Start'].strftime('%Y-%m-%d')} até {camp['End'].strftime('%Y-%m-%d')} ({duration} dias)")

# Função para atribuir campanha a uma data específica
def get_campaign_for_date(date, campaigns_list):
    for camp in campaigns_list:
        if camp['Start'] <= date <= camp['End']:
            return camp['Campanha']
    return 'No Campaign'

# Criar DataFrame inicial
df = pd.DataFrame({
    'Id': range(1, total_rows + 1),
    'Data': dates,
    'Campanha': [get_campaign_for_date(d, campaigns_list) for d in dates],
    'Conjunto de Anúncios': [f'Conjunto_{np.random.randint(1, 40)}' for _ in range(total_rows)],
    'Anúncio': [f'Anuncio_{np.random.randint(1, 100)}' for _ in range(total_rows)],
    'Plataforma': np.random.choice(['Instagram Stories', 'Instagram Feed', 'Explore'], size=total_rows)
})

# Gerar métricas básicas
impressoes = np.random.randint(10000, 20001, size=total_rows)
df['Impressões'] = impressoes
df['Alcance'] = (impressoes * np.random.uniform(0.6, 0.8, size=total_rows)).astype(int)
df['Frequência'] = np.random.uniform(1.5, 2.5, size=total_rows).round(1)

cliques = (impressoes * np.random.uniform(0.01, 0.02, size=total_rows)).astype(int)
df['Cliques no link'] = cliques
df['CTR (%)'] = (cliques / impressoes * 100).round(2)
df['CPC (R$)'] = np.random.uniform(1.5, 3.0, size=total_rows).round(2)

cpm = np.random.uniform(15, 30, size=total_rows).round(2)
df['CPM (R$)'] = cpm
df['Investimento (R$)'] = (impressoes * cpm / 1000).round(2)
df['Conversões'] = (cliques * np.random.uniform(0.01, 0.03, size=total_rows)).astype(int)
df['CPA (R$)'] = np.random.uniform(50, 150, size=total_rows).round(2)
df['ROAS'] = np.random.uniform(3, 6, size=total_rows).round(2)

# Métricas de engajamento
df['Curtidas'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.6).astype(int)
df['Comentários'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.2).astype(int)
df['Compartilhamentos'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.1).astype(int)
df['Salvamentos'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.1).astype(int)
df['Cliques no perfil'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.5).astype(int)
df['Mensagens iniciadas'] = (impressoes * np.random.uniform(0.01, 0.03, size=total_rows) * 0.15).astype(int)
df['Visualizações da página de destino'] = (cliques * np.random.uniform(0.8, 0.9, size=total_rows)).astype(int)
df['Tempo médio visualização vídeo (s)'] = np.random.uniform(5, 50, size=total_rows).round(1)
df['ROI'] = np.random.uniform(2, 5, size=total_rows).round(2)

# Ordenar por data
df = df.sort_values('Data').reset_index(drop=True)

# Gerar métricas diárias com conversão realística (10-15%)
df['Data'] = pd.to_datetime(df['Data'])
daily_data = df.groupby('Data').first().reset_index()

# Função para gerar dados com taxa de conversão realística
def generate_realistic_metrics(n_samples, target_correlation=0.75, conversion_rate_range=(0.10, 0.15)):
    """
    Gera visitas ao perfil e ganho de seguidores com taxa de conversão realística
    """
    max_attempts = 50

    for attempt in range(max_attempts):
        # PRIMEIRO: Gerar o número de novos seguidores por dia (variável dependente)
        # Range mais realístico para 8 anos: 200k -> 680k = 480k ganho total
        # 480k ÷ 2923 dias ≈ 164 seguidores/dia em média
        # Range diário: 50 a 300 seguidores (com média ~164)
        followers_gain = np.random.normal(164, 50, n_samples)  # média 164, desvio 50
        followers_gain = np.clip(followers_gain, 50, 300)  # limitar entre 50-300
        followers_gain = followers_gain.astype(int)

        # Adicionar alguns dias com perdas ocasionais (5% dos dias)
        loss_days = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
        followers_gain[loss_days] = np.random.randint(-20, 0, size=len(loss_days))

        # SEGUNDO: Calcular visitas baseadas na taxa de conversão desejada
        # Se taxa de conversão = 12%, então visitas = seguidores / 0.12
        conversion_rates = np.random.uniform(conversion_rate_range[0], conversion_rate_range[1], n_samples)

        # Para dias com ganho positivo
        positive_mask = followers_gain > 0
        visits = np.zeros(n_samples)

        # Dias com ganho positivo: visitas = ganho / taxa_conversão
        visits[positive_mask] = followers_gain[positive_mask] / conversion_rates[positive_mask]

        # Dias com perda: ainda há visitas, mas sem conversão ou conversão negativa
        negative_mask = followers_gain <= 0
        visits[negative_mask] = np.random.uniform(100, 400, size=negative_mask.sum())

        visits = visits.astype(int)

        # TERCEIRO: Adicionar correlação entre visitas e ganho de seguidores
        # Normalizar ambas as variáveis
        visits_norm = (visits - np.mean(visits)) / np.std(visits)
        followers_norm = (followers_gain - np.mean(followers_gain)) / np.std(followers_gain)

        # Calcular correlação atual
        current_correlation = np.corrcoef(visits_norm, followers_norm)[0, 1]

        # Se correlação está no range aceitável, retornar
        if 0.65 <= current_correlation <= 0.85:
            actual_conversion_rate = np.mean(followers_gain[positive_mask] / visits[positive_mask])
            return visits, followers_gain, current_correlation, actual_conversion_rate

        # Ajustar se necessário
        if current_correlation < 0.65:
            # Aumentar correlação: fazer visitas mais dependentes dos seguidores
            noise_factor = 0.3
            visits_adjusted = followers_gain * 8 + np.random.normal(0, 200, n_samples)  # ~8 visitas por seguidor
            visits_adjusted = np.maximum(visits_adjusted, 100)  # mínimo 100 visitas
            visits = visits_adjusted.astype(int)

    # Se não conseguiu após todas as tentativas, usar método determinístico
    print("Usando método determinístico para garantir taxa de conversão realística...")

    # Método mais direto e controlado
    followers_gain = np.random.normal(164, 50, n_samples)
    followers_gain = np.clip(followers_gain, 50, 300).astype(int)

    # Alguns dias com perdas
    loss_days = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
    followers_gain[loss_days] = np.random.randint(-20, 0, size=len(loss_days))

    # Calcular visitas para taxa de conversão de ~12%
    base_conversion = 0.12
    visits = np.zeros(n_samples)

    positive_mask = followers_gain > 0
    # Para dias positivos: visitas = seguidores / 0.12 + ruído
    visits[positive_mask] = (followers_gain[positive_mask] / base_conversion +
                           np.random.normal(0, 100, positive_mask.sum()))

    # Para dias negativos: visitas aleatórias entre 200-500
    negative_mask = followers_gain <= 0
    visits[negative_mask] = np.random.uniform(200, 500, negative_mask.sum())

    visits = np.maximum(visits, 100).astype(int)  # mínimo 100 visitas

    correlation_achieved = np.corrcoef(visits, followers_gain)[0, 1]
    actual_conversion_rate = np.mean(followers_gain[positive_mask] / visits[positive_mask])

    return visits, followers_gain, correlation_achieved, actual_conversion_rate

# Gerar dados com taxa de conversão realística
n_days = len(daily_data)
print(f"Gerando métricas para {n_days} dias...")

visits, followers_gain, correlation, conversion_rate = generate_realistic_metrics(n_days)

daily_data['Visitas ao perfil'] = visits
daily_data['Nº Seguidores Dia'] = followers_gain

print(f"Taxa de conversão média: {conversion_rate:.3f} ({conversion_rate*100:.1f}%)")
print(f"Correlação obtida: {correlation:.3f}")

# Calcular seguidores acumulados
initial_followers = 200000
daily_data['Nº Seguidores'] = initial_followers + daily_data['Nº Seguidores Dia'].cumsum()

# Verificar se atingiu o target de 680k seguidores
final_followers = daily_data['Nº Seguidores'].iloc[-1]
target_followers = 680000

print(f"Seguidores finais: {final_followers:,}")

# Se necessário, ajustar para atingir target mantendo taxa de conversão
if abs(final_followers - target_followers) > 10000:  # se diferença > 10k
    print(f"Ajustando para atingir target de {target_followers:,} seguidores...")

    total_gain_needed = target_followers - initial_followers
    total_gain_current = daily_data['Nº Seguidores Dia'].sum()
    adjustment_factor = total_gain_needed / total_gain_current

    print(f"Fator de ajuste: {adjustment_factor:.3f}")

    # Ajustar ganho de seguidores
    adjusted_followers_gain = daily_data['Nº Seguidores Dia'] * adjustment_factor

    # Manter taxas de conversão realísticas ajustando visitas proporcionalmente
    positive_mask = adjusted_followers_gain > 0
    adjusted_visits = daily_data['Visitas ao perfil'].copy()

    # Para dias positivos, ajustar visitas mantendo a taxa de conversão
    if positive_mask.sum() > 0:
        current_rates = daily_data.loc[positive_mask, 'Nº Seguidores Dia'] / daily_data.loc[positive_mask, 'Visitas ao perfil']
        adjusted_visits[positive_mask] = adjusted_followers_gain[positive_mask] / current_rates

    # Aplicar ajustes
    daily_data['Nº Seguidores Dia'] = adjusted_followers_gain.astype(int)
    daily_data['Visitas ao perfil'] = adjusted_visits.astype(int)
    daily_data['Nº Seguidores'] = initial_followers + daily_data['Nº Seguidores Dia'].cumsum()

    # Recalcular métricas
    final_followers = daily_data['Nº Seguidores'].iloc[-1]
    correlation = np.corrcoef(daily_data['Visitas ao perfil'], daily_data['Nº Seguidores Dia'])[0, 1]

    positive_days = daily_data['Nº Seguidores Dia'] > 0
    if positive_days.sum() > 0:
        conversion_rate = np.mean(daily_data.loc[positive_days, 'Nº Seguidores Dia'] /
                                daily_data.loc[positive_days, 'Visitas ao perfil'])

    print(f"Seguidores finais após ajuste: {final_followers:,}")
    print(f"Taxa de conversão após ajuste: {conversion_rate:.3f} ({conversion_rate*100:.1f}%)")
    print(f"Correlação após ajuste: {correlation:.3f}")

# Distribuir métricas diárias entre os anúncios do dia
print("Distribuindo métricas diárias entre anúncios...")

# Contar quantos anúncios por dia
ads_per_day = df.groupby('Data').size().reset_index(name='ads_count')
daily_data = daily_data.merge(ads_per_day, on='Data', how='left')

# Função para distribuir valores de forma proporcional com variação
def distribute_daily_value(total_value, n_ads, min_share=0.05, max_share=0.25):
    """
    Distribui um valor total entre n_ads de forma proporcional com variação
    """
    if n_ads == 1:
        return [total_value]

    # Gerar pesos aleatórios
    weights = np.random.uniform(min_share, max_share, n_ads)
    weights = weights / weights.sum()  # Normalizar para somar 1

    # Distribuir o valor total
    distributed = (total_value * weights).astype(int)

    # Ajustar para que a soma seja exata
    diff = total_value - distributed.sum()
    if diff != 0:
        # Adicionar/subtrair a diferença no maior valor
        max_idx = np.argmax(distributed)
        distributed[max_idx] += diff

    return distributed

# Criar DataFrames para armazenar distribuições
distributed_data = []

for _, day_data in daily_data.iterrows():
    date = day_data['Data']
    n_ads = day_data['ads_count']

    # Distribuir seguidores do dia
    followers_dist = distribute_daily_value(day_data['Nº Seguidores Dia'], n_ads)

    # Distribuir visitas do dia
    visits_dist = distribute_daily_value(day_data['Visitas ao perfil'], n_ads)

    # Armazenar resultados
    for i in range(n_ads):
        distributed_data.append({
            'Data': date,
            'ad_index': i,
            'Nº Seguidores Dia': followers_dist[i],
            'Visitas ao perfil': visits_dist[i]
        })

# Converter para DataFrame
dist_df = pd.DataFrame(distributed_data)

# Criar índice para ordenação no DataFrame principal
df_sorted = df.sort_values(['Data', 'Id']).reset_index(drop=True)
df_sorted['ad_index'] = df_sorted.groupby('Data').cumcount()

# Fazer merge com dados distribuídos
df_sorted = df_sorted.merge(
    dist_df[['Data', 'ad_index', 'Nº Seguidores Dia', 'Visitas ao perfil']],
    on=['Data', 'ad_index'],
    how='left'
)

# Adicionar seguidores acumulados
# Primeiro, calcular total diário real
daily_totals = df_sorted.groupby('Data')['Nº Seguidores Dia'].sum().reset_index()
daily_totals['Nº Seguidores'] = initial_followers + daily_totals['Nº Seguidores Dia'].cumsum()

# Adicionar seguidores acumulados ao DataFrame principal
daily_totals_renamed = daily_totals.rename(columns={'Nº Seguidores': 'Nº Seguidores Acumulado'})
df_sorted = df_sorted.merge(daily_totals_renamed[['Data', 'Nº Seguidores Acumulado']], on='Data', how='left')
df_sorted['Nº Seguidores'] = df_sorted['Nº Seguidores Acumulado']
df_sorted = df_sorted.drop('Nº Seguidores Acumulado', axis=1)

# Remover coluna auxiliar
df_sorted = df_sorted.drop('ad_index', axis=1)

# Atualizar DataFrame principal
df = df_sorted

# Verificações finais
print("\n=== VERIFICAÇÕES FINAIS ===")

# Calcular estatísticas da taxa de conversão
daily_summary = df.groupby('Data').agg({
    'Visitas ao perfil': 'sum',
    'Nº Seguidores Dia': 'sum'
}).reset_index()

positive_days = daily_summary['Nº Seguidores Dia'] > 0
conversion_rates_daily = daily_summary.loc[positive_days, 'Nº Seguidores Dia'] / daily_summary.loc[positive_days, 'Visitas ao perfil']

print(f"Taxa de conversão média: {conversion_rates_daily.mean():.3f} ({conversion_rates_daily.mean()*100:.1f}%)")
print(f"Taxa de conversão mediana: {conversion_rates_daily.median():.3f} ({conversion_rates_daily.median()*100:.1f}%)")
print(f"Range da taxa de conversão: {conversion_rates_daily.min():.3f} - {conversion_rates_daily.max():.3f}")

# Verificar correlação final
final_correlation = np.corrcoef(daily_summary['Visitas ao perfil'], daily_summary['Nº Seguidores Dia'])[0, 1]
print(f"Correlação final: {final_correlation:.3f}")

# Verificar crescimento
print(f"Seguidores iniciais: {initial_followers:,}")
print(f"Seguidores finais: {daily_summary['Nº Seguidores Dia'].cumsum().iloc[-1] + initial_followers:,}")
print(f"Crescimento total: {daily_summary['Nº Seguidores Dia'].sum():,} seguidores")

# Estatísticas de visitas e seguidores
print(f"\nVisitas diárias - Média: {daily_summary['Visitas ao perfil'].mean():.0f}, Mediana: {daily_summary['Visitas ao perfil'].median():.0f}")
print(f"Seguidores diários - Média: {daily_summary['Nº Seguidores Dia'].mean():.0f}, Mediana: {daily_summary['Nº Seguidores Dia'].median():.0f}")

# Verificar se há valores nulos
print(f"\nValores nulos no DataFrame: {df.isnull().sum().sum()}")

# Salvar arquivo
df.to_csv('adjusted_metrics.csv', index=False, encoding='utf-8-sig')
print(f"\nArquivo salvo como 'realistic_conversion_metrics_8years.csv'")

# Amostra dos dados
print("\nPrimeiras 5 linhas:")
print(df[['Data', 'Campanha', 'Nº Seguidores', 'Nº Seguidores Dia', 'Visitas ao perfil']].head())

# Verificar algumas taxas de conversão por linha para validação
sample_data = df[df['Nº Seguidores Dia'] > 0].head()
if len(sample_data) > 0:
    print("\nTaxas de conversão em algumas linhas (apenas dias positivos):")
    for _, row in sample_data.iterrows():
        conv_rate = row['Nº Seguidores Dia'] / row['Visitas ao perfil']
        print(f"Data: {row['Data'].strftime('%Y-%m-%d')}, Visitas: {row['Visitas ao perfil']}, Seguidores: {row['Nº Seguidores Dia']}, Taxa: {conv_rate:.3f} ({conv_rate*100:.1f}%)")

Campanhas geradas:
Campanha_1: 2017-01-01 até 2017-05-14 (133 dias)
Campanha_2: 2017-05-15 até 2017-10-09 (147 dias)
Campanha_3: 2017-10-10 até 2018-02-08 (121 dias)
Campanha_4: 2018-02-09 até 2018-04-29 (79 dias)
Campanha_5: 2018-04-30 até 2018-07-23 (84 dias)
Campanha_6: 2018-07-24 até 2018-10-15 (83 dias)
Campanha_7: 2018-10-16 até 2019-01-02 (78 dias)
Campanha_8: 2019-01-03 até 2019-03-29 (85 dias)
Campanha_9: 2019-03-30 até 2019-06-23 (85 dias)
Campanha_10: 2019-06-24 até 2019-10-15 (113 dias)
Campanha_11: 2019-10-16 até 2020-03-05 (141 dias)
Campanha_12: 2020-03-06 até 2020-06-26 (112 dias)
Campanha_13: 2020-06-27 até 2020-09-29 (94 dias)
Campanha_14: 2020-09-30 até 2020-12-13 (74 dias)
Campanha_15: 2020-12-14 até 2021-02-12 (60 dias)
Campanha_16: 2021-02-13 até 2021-06-20 (127 dias)
Campanha_17: 2021-06-21 até 2021-09-18 (89 dias)
Campanha_18: 2021-09-19 até 2022-02-24 (158 dias)
Campanha_19: 2022-02-25 até 2022-06-21 (116 dias)
Campanha_20: 2022-06-22 até 2022-09-25 (95 dias)
C

/tmp/ipython-input-9-2152662194.py:247: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1712.57468804 2055.08962565 1707.15514789 ... 1832.88847941 2140.71836005
 1060.06205373]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  adjusted_visits[positive_mask] = adjusted_followers_gain[positive_mask] / current_rates



=== VERIFICAÇÕES FINAIS ===
Taxa de conversão média: 0.121 (12.1%)
Taxa de conversão mediana: 0.120 (12.0%)
Range da taxa de conversão: 0.084 - 0.273
Correlação final: 0.966
Seguidores iniciais: 200,000
Seguidores finais: 678,703
Crescimento total: 478,703 seguidores

Visitas diárias - Média: 1419, Mediana: 1458
Seguidores diários - Média: 167, Mediana: 173

Valores nulos no DataFrame: 0

Arquivo salvo como 'realistic_conversion_metrics_8years.csv'

Primeiras 5 linhas:
        Data    Campanha  Nº Seguidores  Nº Seguidores Dia  Visitas ao perfil
0 2017-01-01  Campanha_1         200218                 17                331
1 2017-01-01  Campanha_1         200218                 18                165
2 2017-01-01  Campanha_1         200218                 41                239
3 2017-01-01  Campanha_1         200218                 22                 86
4 2017-01-01  Campanha_1         200218                 36                 86

Taxas de conversão em algumas linhas (apenas dias positi